In [3]:
import pandas as pd # Dataframes and file IO
import numpy as np # numerical calculations


In [4]:
# Import from Bokeh python library for vizualisation
from bokeh.models import HoverTool, TapTool, ColumnDataSource 
from bokeh.models.widgets import DataTable, DateFormatter, TableColumn

from bokeh.layouts import row
from bokeh.plotting import figure
from bokeh.io import output_file, show


In [5]:
df=pd.read_csv('input_files/Bokeh_volcano_dataframe.csv')  #import data without MGI symbol as key
df3 = df[df['q-value'] < 0.01]
df3.head()   # check table

MGI symbol     Ensembl Gene ID  \
0       Rin1  ENSMUSG00000024883   
1       Asb4  ENSMUSG00000042607   
2   Arhgap36  ENSMUSG00000036198   
3    Slc30a3  ENSMUSG00000029151   
4      Ramp3  ENSMUSG00000041046   

                                         Description  MoEx_Enrich  \
0  Ras and Rab interactor 1 [Source:MGI Symbol;Ac...        -1.68   
1  ankyrin repeat and SOCS box-containing 4 [Sour...         2.55   
2  Rho GTPase activating protein 36 [Source:MGI S...         3.20   
3  solute carrier family 30 (zinc transporter), m...        -3.01   
4  receptor (calcitonin) activity modifying prote...        -1.66   

   MoEx_Hedges_g  MoEx_Var_g  MoEx_SEg  MoGene_Enrich  MoGene_Hedges_g  \
0          -8.19        1.40      1.18          -1.57            -6.87   
1           5.03        0.65      0.81           3.27             5.42   
2           7.04        1.09      1.04           3.57             6.93   
3          -7.56        1.22      1.11          -3.60            -8.66   
4          -5.76        0.79      0.89          -1.39            -5.41   

   MoGene_Var_g  ...       esW*5  REM_M*  REM_SEM*      REM_Z      REM_pVal  \
0          1.14  ...   -4.207114   -6.79  0.540770 -12.547917  4.080000e-36   
1          0.77  ...    4.361766    5.64  0.470262  11.991067  3.960000e-33   
2          1.16  ...    3.929354    7.24  0.603795  11.989094  4.050000e-33   
3          1.72  ...   -3.830472   -6.97  0.584324 -11.923629  8.910000e-33   
4          0.77  ...   -5.463157   -5.19  0.435443 -11.919953  9.320000e-33   

        q-value      lfdr  color  alpha  log_q  
0  8.444940e-32  0.000001   navy    0.4 -31.07  
1  2.795730e-29  0.000001   gold    0.4 -28.55  
2  2.795730e-29  0.000001   gold    0.4 -28.55  
3  3.855540e-29  0.000001   navy    0.4 -28.41  
4  3.855540e-29  0.000001   navy    0.4 -28.41  

[5 rows x 67 columns]

In [6]:
def plot_and_table(df):


    output_file(filename="plot_plus_table.html", title="Figure2_with_table")


    p3 = figure(tools="wheel_zoom,crosshair,pan,box_zoom,reset,previewsave,box_select, hover, tap",
               x_range=(-20, 15), y_range=(-35, 0), output_backend = "webgl")


    source = ColumnDataSource(data = dict(x=df3['REM_M*'].round(2), 
                                          y= df3['log_q'].round(2),
                                          c =df3['color'],
                                          a = df3['alpha'],
                                          studies = df3['df']+1,
                                          radius = (df3['df']+1)/16,
                                          names = df3['MGI symbol'],
                                          I2 = df3['I^2'].round(2)))


    p3.scatter('x' , 'y' , source = source, radius='radius',
             fill_alpha = 'a', fill_color = 'c',line_alpha = 0.5, line_color = 'black', line_width=0.2)


    p3.xaxis.axis_label = "Combined Effect Size (M*)"
    p3.yaxis.axis_label = "log10(q value)"

    hover = p3.select(dict(type=HoverTool))
    hover.tooltips = [("ID", "@names"),
                      ("Number of Studies", "@studies"),
                      ("q value(log10)","@y"),
                      ("Effect size (M)*","@x"),
                      ("I-Squared","@I2"),]



    p3.ygrid.band_fill_color=None
    p3.ygrid.grid_line_color=None
    p3.xgrid.band_fill_color=None
    p3.xgrid.grid_line_color=None
    p3.line(x= 0, y= [-35,0], line_width=4, line_dash=[8,8], line_color="grey")
    p3.line(x= -3, y= [-35,0], line_width=2, line_dash=[8,8], line_color="navy")
    p3.line(x= 3, y= [-35,0], line_width=2, line_dash=[8,8], line_color="gold")
    p3.line(x= [-20,20], y= -2, line_width=1, line_dash=[8,8], line_color="grey")


    taptool = p3.select(type=TapTool)


    columns = [TableColumn(field="names", title="MGI symbol"),
               TableColumn(field="studies", title="Studies"),
               TableColumn(field="x", title="Effect Size"),
               TableColumn(field="y", title="q-value"),
               TableColumn(field="I2", title="I-Squared")]


    data_table = DataTable(source=source, columns=columns, height=800, width=600)


    show(row(data_table,p3)) #

In [7]:
plot_and_table(df3)